# GROUP 48 || Project : Moving Rating Predictor

### *We extracted the links, movie_medtadata and credits dataset from TMDB and ratings dataset was extracted from MovieLens. The links.csv file consisted of movieId, imdbId and tmdbId. The movies_metadata.csv file contained the most important content regarding budget of the movie, movie title, movie genre, imdbId, revenue, runtime, vote average, vote count, popularity of a movie, etc. The ratings.csv file provides information regarding userId, movieId, rating of a movie. The credits.csv file highlight information about cast, crew and id of a movie*

In [84]:
#this our data cleaning notebook, we are going to merge our three working datasets into one. Also in this notebook we will make
# a decision on NaN values


In [85]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [6]:
linksdf = pd.read_csv('data/links.csv')
moviedf = pd.read_csv('data/movies_metadata.csv')
ratingsdf = pd.read_csv('data/ratings.csv')
credits = pd.read_csv('data/credits.csv')

In [7]:
print('Shape of links data:', linksdf.shape[0])
print('shape of movie data:', moviedf.shape[0])
print('shape of ratings data:', ratingsdf.shape[0])

Shape of links data: 45843
shape of movie data: 45466
shape of ratings data: 26024289


# Data Cleaning

### *Creating castingdf by obtaining actors and directors information from credits.csv file*

### *Getting actors from the credits file*

In [8]:

credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [9]:
import ast
credits['cast'] = credits['cast'].apply(ast.literal_eval)

In [10]:
credits['cast_size'] = credits['cast'].apply(lambda x: len(x))


In [11]:
credits['cast'] = credits['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [12]:
s = credits.apply(lambda x: pd.Series(x['cast']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'actor'
cast_df = credits.drop('cast', axis=1).join(s)

<ipython-input-12-c4dc3434c0f3>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = credits.apply(lambda x: pd.Series(x['cast']),axis=1).stack().reset_index(level=1, drop=True)


In [13]:
castingdf = cast_df.drop_duplicates(subset=['id'], keep='first')

In [14]:
castingdf.head()

,crew,id,cast_size,actor
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,13,Tom Hanks
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,26,Robin Williams
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,7,Walter Matthau
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,10,Whitney Houston
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,12,Steve Martin


### *Getting directors from the credits file*

In [15]:
castingdf['crew'] = castingdf['crew'].apply(ast.literal_eval)

<ipython-input-15-2ec3df0fc930>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  castingdf['crew'] = castingdf['crew'].apply(ast.literal_eval)


In [16]:
castingdf['crew_size'] = castingdf['crew'].apply(lambda x: len(x))

<ipython-input-16-08f46432f65d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  castingdf['crew_size'] = castingdf['crew'].apply(lambda x: len(x))


In [17]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [18]:
castingdf['director'] = castingdf['crew'].apply(get_director)

<ipython-input-18-fb440a245827>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  castingdf['director'] = castingdf['crew'].apply(get_director)


In [19]:
castingdf.head()

,crew,id,cast_size,actor,crew_size,director
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,13,Tom Hanks,106,John Lasseter
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,26,Robin Williams,16,Joe Johnston
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,7,Walter Matthau,4,Howard Deutch
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,10,Whitney Houston,10,Forest Whitaker
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,12,Steve Martin,7,Charles Shyer


In [20]:
col_name="id"
first_col = castingdf.pop(col_name)
castingdf.insert(0, col_name, first_col)

### *Checking whether the changes were properly executed on dataset*

In [21]:
castingdf

,id,crew,cast_size,actor,crew_size,director
0,862,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",13,Tom Hanks,106,John Lasseter
1,8844,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",26,Robin Williams,16,Joe Johnston
2,15602,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",7,Walter Matthau,4,Howard Deutch
3,31357,"[{'credit_id': '52fe44779251416c91011acb', 'de...",10,Whitney Houston,10,Forest Whitaker
4,11862,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",12,Steve Martin,7,Charles Shyer
...,...,...,...,...,...,...
45471,439050,"[{'credit_id': '5894a97d925141426c00818c', 'de...",3,Leila Hatami,9,Hamid Nematollah
45472,111109,"[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",11,Angel Aquino,6,Lav Diaz
45473,67758,"[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",15,Erika Eleniak,5,Mark L. Lester
45474,227506,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",5,Iwan Mosschuchin,2,Yakov Protazanov


### *Merging casting and movies data set*

In [22]:
moviedf.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [23]:
col_name="id"
first_col = moviedf.pop(col_name)
moviedf.insert(0, col_name, first_col)

In [24]:
#keeping only the relevant columns

columns = list(moviedf.columns)
maincolum = ['genres', 'id','budget','title','overview', 'revenue', 'release_date', 'runtime', 'adult', 'original_language','production_companies','vote_average','vote_count','production_countries','belongs_to_collection','homepage']

list1= []
for i in columns:
    if i not in maincolum:
        list1.append(i)
moviesdf = moviedf.drop(list1, axis=1)

In [25]:
moviesdf.isna().sum()

id                           0
adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
original_language           11
overview                   954
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
title                        6
vote_average                 6
vote_count                   6
dtype: int64

### *Dropping all the null values from moviesdf to get clean version of the data*

In [26]:
#dropping NaN values
moviesdf.isna().sum()
cleandf = moviesdf.dropna(subset=['genres', 'id','budget','title','overview', 'revenue', 'release_date', 'runtime', 'adult', 'original_language','production_companies','vote_average','vote_count','production_countries'])

In [27]:
cleandf.isna().sum()

id                           0
adult                        0
belongs_to_collection    39996
budget                       0
genres                       0
homepage                 36676
original_language            0
overview                     0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                      0
title                        0
vote_average                 0
vote_count                   0
dtype: int64

In [28]:
cleandf.shape[0]

44425

In [29]:
#converting ID into int
cleandf['id'] = cleandf.id.astype('int64')

<ipython-input-29-364ac506764b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf['id'] = cleandf.id.astype('int64')


### *Creating workding1df by merging Cleandf and casting df with reference to id*

In [30]:
workingdf = pd.merge(cleandf, castingdf, on='id')

In [31]:
working1df = workingdf.dropna(subset=['genres', 'id','budget','title','overview', 'revenue', 'release_date', 'runtime', 'adult', 'original_language','production_companies','vote_average','vote_count','production_countries'])
working1df.shape

(44424, 21)

# Working on links data set

### *Dropping Na values*

In [32]:
#cleaning links dataset
cleanlinksdf = linksdf.dropna()

### *Renaming column name TMDB id as ID in order to merge it with working1df*

In [33]:
#renaming TMDB id as ID to make it consistent with links dataset
cleanlinksdf.rename(columns = {'tmdbId':'id' }, inplace = True)


C:\Users\admin\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [34]:
#converting the id into int
cleanlinksdf.id = cleanlinksdf.id.astype('int64')

C:\Users\admin\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [35]:
cleanlinksdf.head()

,movieId,imdbId,id
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


In [36]:
col_name="id"
first_col = cleanlinksdf.pop(col_name)
cleanlinksdf.insert(0, col_name, first_col)

In [37]:
cleanlinksdf.head()

,id,movieId,imdbId
0,862,1,114709
1,8844,2,113497
2,15602,3,113228
3,31357,4,114885
4,11862,5,113041


In [38]:
#merging two datasets on id
tempdf = pd.merge(working1df,cleanlinksdf,on='id')

In [39]:
#dropping ImbdbID column
temp1df = tempdf.drop(['imdbId'], axis=1)

### *We next filter the data so that only our target genre labels are included and we assign each text to just one of the genre labels. It's possible that one text could be labelled with two of these labels (eg. Science Fiction and Fantasy) but we will just assign one of those here*

In [40]:
target_genres = ["Animation",
                 'Adventure',
                 'Romance',
                 'Comedy',
                 'Drama',
                'Horror', 'Science Fiction', 'Fantasy', 'Crime', 'Action', 'Mystery', 'Family', 'Music', 'Western', 'War', 'Thriller']

# create a Series of empty strings the same length as the list of movies
genre = pd.Series(np.repeat("", temp1df.shape[0]))
# look for each target genre and set the corresponding entries in the genre series to the genre label
for g in target_genres:
    genre[temp1df['genres'].str.contains(g)] = g

# add this to the movie dataframe and then select only those rows that have a genre label

temp1df['genres'] = genre
genre_movies = temp1df[genre!='']

genre_movies.shape

(39376, 22)

In [41]:
#converting budget into float and getting a profit column

genre_movies.budget = genre_movies.budget.astype('float64')
genre_movies['profit'] = genre_movies.revenue - genre_movies.budget

C:\Users\admin\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-41-cacc4cb4ec51>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_movies['profit'] = genre_movies.revenue - genre_movies.budget


### *Obtaining Fulldf by merging movies and rating*

In [42]:
#moving movieID column to the first column
col_name="movieId"
first_col = genre_movies.pop(col_name)
genre_movies.insert(0, col_name, first_col)

In [43]:
#moving movieID column to the first column in our ratings dataset
col_name1="movieId"
first_col1 = ratingsdf.pop(col_name)
ratingsdf.insert(0, col_name1, first_col1)

In [44]:
ratingsdf.head()

,movieId,userId,rating,timestamp
0,110,1,1.0,1425941529
1,147,1,4.5,1425942435
2,858,1,5.0,1425941523
3,1221,1,5.0,1425941546
4,1246,1,5.0,1425941556


In [45]:
#complete dataframe
fulldf = pd.merge(ratingsdf,genre_movies,on='movieId')


### *Obtaining finaldf by dropping timestamp*

In [46]:
#final dataset after dropping unwanted columns
finaldf = fulldf.drop(['timestamp'], axis=1)
print('Shape of final dataset', finaldf.shape[0])

Shape of final dataset 25765731


### *Dropping rows with no profit values*

In [47]:
index1 = finaldf[finaldf.profit == 0.0 ].index

In [48]:
final1df = finaldf.drop(index1, axis=0)

In [49]:
final1df.shape[0]

22986508

In [50]:
listMovie = list(final1df.movieId) 

### *Evaluating average ratings for a particular movie*

In [51]:
listAverage = [] 
for i in set(listMovie):
    listAverage.append(int(round(final1df[final1df.movieId == i].rating.mean(),1)))
    

In [52]:
final1df.shape

(22986508, 25)

In [53]:
final1df['actor'] = final1df['actor'].astype(str)

In [54]:
final1df['director'] = final1df['director'].astype(str)

In [55]:
final1df.dtypes

movieId                    int64
userId                     int64
rating                   float64
id                         int64
adult                     object
belongs_to_collection     object
budget                   float64
genres                    object
homepage                  object
original_language         object
overview                  object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
title                     object
vote_average             float64
vote_count               float64
crew                      object
cast_size                  int64
actor                     object
crew_size                  int64
director                  object
profit                   float64
dtype: object

In [56]:
final2df = final1df.drop_duplicates(subset=['movieId'])

In [57]:
final2df.shape

(10478, 25)

In [58]:

x = pd.DataFrame(columns= ['movieId'],data= list(set(listMovie)))
y = pd.DataFrame(columns=['average_ratings'], data=listAverage)



In [59]:
type(x)

pandas.core.frame.DataFrame

In [60]:
ratingaverage =[x,y]

In [61]:
ratingaverage[1]

,average_ratings
0,3
1,3
2,2
3,3
4,3
...,...
10473,3
10474,3
10475,4
10476,0


In [62]:
ratingaverage[0]

,movieId
0,1
1,2
2,4
3,5
4,6
...,...
10473,98273
10474,32743
10475,65514
10476,131062


### *Mapping average rating with movieId*

In [63]:
ratingAverageDF = x.join(y,how = 'inner' )


In [64]:
ratingAverageDF

,movieId,average_ratings
0,1,3
1,2,3
2,4,2
3,5,3
4,6,3
...,...,...
10473,98273,3
10474,32743,3
10475,65514,4
10476,131062,0


In [65]:
ratingAverageDF 

,movieId,average_ratings
0,1,3
1,2,3
2,4,2
3,5,3
4,6,3
...,...,...
10473,98273,3
10474,32743,3
10475,65514,4
10476,131062,0


In [66]:
final2df.shape[0]

10478

### *Merging ratingAverageDF with finaldf*

In [67]:
final3df = pd.merge(final2df, ratingAverageDF, on='movieId')

In [68]:
final3df[final3df.movieId == 1]

,movieId,userId,rating,id,adult,belongs_to_collection,budget,genres,homepage,original_language,...,title,vote_average,vote_count,crew,cast_size,actor,crew_size,director,profit,average_ratings
176,1,8,4.0,862,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,Family,http://toystory.disney.com/toy-story,en,...,Toy Story,7.7,5415.0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",13,Tom Hanks,106,John Lasseter,343554033.0,3


In [69]:
final2df

,movieId,userId,rating,id,adult,belongs_to_collection,budget,genres,homepage,original_language,...,runtime,title,vote_average,vote_count,crew,cast_size,actor,crew_size,director,profit
0,110,1,1.0,197,False,NaN,72000000.0,War,NaN,en,...,177.0,Braveheart,7.7,3404.0,"[{'credit_id': '52fe4225c3a36847f80079c1', 'de...",55,Mel Gibson,67,Mel Gibson,138000000.0
71479,858,1,5.0,238,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000.0,Crime,http://www.thegodfather.com/,en,...,175.0,The Godfather,8.5,6024.0,"[{'credit_id': '52fe422bc3a36847f80093db', 'de...",58,Marlon Brando,42,Francis Ford Coppola,239066411.0
128549,1221,1,5.0,240,False,"{'id': 230, 'name': 'The Godfather Collection'...",13000000.0,Crime,NaN,en,...,200.0,The Godfather: Part II,8.3,3418.0,"[{'credit_id': '52fe422bc3a36847f80094dd', 'de...",84,Al Pacino,53,Francis Ford Coppola,34542841.0
165228,1246,1,5.0,207,False,NaN,16400000.0,Drama,NaN,en,...,129.0,Dead Poets Society,8.1,2786.0,"[{'credit_id': '52fe4227c3a36847f8008045', 'de...",15,Robin Williams,11,Peter Weir,219460116.0
190980,1968,1,4.0,2108,False,NaN,1000000.0,Drama,NaN,en,...,97.0,The Breakfast Club,7.8,2189.0,"[{'credit_id': '52fe4336c3a36847f8042e63', 'de...",13,Emilio Estevez,14,John Hughes,50525171.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25765703,161030,270123,2.0,93091,False,NaN,10000000.0,Drama,NaN,en,...,95.0,Crazy Eyes,4.3,5.0,"[{'credit_id': '52fe491b9251416c750bdac1', 'de...",4,Lukas Haas,3,Adam Sherman,-10000000.0
25765716,136928,270887,3.0,290556,False,NaN,10000000.0,Thriller,http://www.saradanmedia.co.uk/films/viktor/,ru,...,134.0,Viktor,5.0,2.0,"[{'credit_id': '54409fb10e0a2628d9000aa5', 'de...",5,Gérard Depardieu,2,Philippe Martinez,-10000000.0
25765722,145078,270887,1.0,59095,False,NaN,0.0,Fantasy,NaN,fr,...,0.0,Revelation,2.9,9.0,"[{'credit_id': '52fe4981c3a36847f819d97d', 'de...",9,James D'Arcy,8,Stuart Urban,10000000.0
25765727,159053,270887,5.0,56491,False,NaN,1000000.0,Thriller,NaN,en,...,88.0,In Her Line of Fire,3.3,9.0,"[{'credit_id': '52fe4902c3a36847f8184ed1', 'de...",14,Mariel Hemingway,4,Brian Trenchard-Smith,-1000000.0


In [70]:
final4df =  final3df.drop(['userId', 'rating'], axis=1)

# Removing budget and revenue = 0

In [71]:
revenueind = final4df[final4df.revenue == 0].index

In [72]:
final5df = final4df.drop(revenueind, axis=0)

In [73]:
budgetind = final5df[final5df.budget == 0].index

In [74]:
final6df = final5df.drop(budgetind, axis=0)

In [75]:
final7df = final6df.drop('crew', axis=1)

In [77]:
final7df.to_csv('FinalAddDf')

In [81]:
ourMaindf = pd.read_csv('FinalAdddf', index_col=0)

### *Evaluating Final Shape of our dataframe*

In [83]:
print('our final working DF shape', ourMaindf.shape[0])

our final working DF shape 5297
